In [10]:
import numpy as np
import matplotlib.pyplot as plt
import powerlaw
import time
import datetime
from sklearn import linear_model
from scipy.stats import norm

In [11]:
def data_form(fin, fout):
    f = open(fin, 'r', encoding='utf8')
    f_out = open(fout, 'w', encoding='utf8')
    pos = 0
    for line in f:
        pos+=1
        if pos%1000000 == 0:
            print(pos)
        c = line.split(',')
        tmp_timestamp = change_time(c[3])
        tmp_timestamp = tmp_timestamp.replace('-','')
        print(c[0], c[1], c[2], tmp_timestamp, file = f_out)
    f.close()
    f_out.close()

def change_time(u):
    u = int(u)
    # u = 1393200000        #unix时间戳
    t = datetime.datetime.fromtimestamp(u)
    t_str = str(t)
    return t_str.split(' ')[0]

def data_processing():
    for csv in ['ratings_Books', 'ratings_CDs_and_Vinyl', 'ratings_Electronics', 'ratings_Movies_and_TV']:
        filename_in = '../RealDatasetValidation/data/' + csv + '.csv'
        new_csv = csv.replace('ratings_', '')
        filename_out = '../RealDatasetValidation/data/amazon/' + new_csv +'.txt'
        print("********************" + new_csv + "*******************")
        print("input", filename_in)
        print("output", filename_out)
        data_form(filename_in, filename_out)
        print("********************END" + new_csv + "*******************")

def sort_by_time(fin, fout):
    f = open(fin, 'r', encoding='utf8')
    f_out = open(fout, 'w', encoding='utf8')
    pos = 0
    dataset = []
    for line in f:
        pos+=1
        if pos%1000000 == 0:
            print(pos)
        c = line.split(' ')
        dataset.append((c[0], c[1], c[2], int(c[3])))
    print("sorting......")
    dataset = sorted(dataset,key=lambda x:x[3])
    for i in range(0, len(dataset)):
        print(dataset[i][0], dataset[i][1], dataset[i][2], str(dataset[i][3]), file=f_out)
    f.close()
    f_out.close()

# 22507155 book
# 7824482 electronic
# 4607047 movie
# 3749004 CD

def graph_main(fin, num, pace):
    stop_time = int(pace * num)
    print('stoptime:', stop_time, "pace:", pace)
    degree = [0]*10000000
    item = [0]*10000000
    weight = [0]*10000000
    # files = 'ratings_Books.csv'
    f = open(fin, 'r', encoding='utf8')
    data = []
    cnt =0
    ytime = 19700101
    print ('Start reading from '+fin)
    for line in f:
        cnt = cnt + 1
        if cnt ==1:
            continue
        if cnt == stop_time:
            ytime = int(line.split(' ')[3])
            print('ytime', ytime)
            break
        if cnt % 1000000 == 0:
            print (cnt)
        line = line.split(' ')
        user = hash(line[0])%10000000
        book = hash(line[1])%10000000
        rate = float(line[2])
        degree[book] = degree[book] + 1
        weight[book] = weight[book] + rate
    
    ###########useless############
    deg = np.array(degree)
    bins = np.bincount(deg)
    np.set_printoptions(threshold=np.inf)
    degseq = bins[:20000]
    xind = np.arange(degseq.shape[0])
    # phi_deg = DataFit(xind[1:100], degseq[1:100])
    
    # plt.plot(xind[1:100],degseq[1:100],'bo')
    # plt.title('User Degree Distributions')
    # plt.xlabel('Degree')
    # plt.ylabel('Number')
    # plt.show()
    # plt.plot(np.log(xind[1:100]),np.log(degseq[1:100]),'bo')
    # plt.title('User Degree Distributions (Logarithm)')
    # plt.xlabel('Degree')
    # plt.ylabel('Number')
    # plt.show()
    wei = np.array(weight)
    binw = np.bincount(wei.astype('int'))
    weiseq = binw[:20000]
    phi_wei = DataFit(xind, weiseq[1:len(xind)])
    # plt.plot(xind[1:100],weiseq[1:100],'ro')
    # plt.title('User Vertex Weight Distributions')
    # plt.xlabel('Vertex weight')
    # plt.ylabel('Number')
    # plt.show()
    # plt.plot(np.log(xind[1:100]),np.log(weiseq[1:100]),'ro')
    # plt.title('User Vertex Weight Distributions (Logarithm)')
    # plt.xlabel('Vertex weight')
    # plt.ylabel('Number')
    # plt.show()
    return phi_wei, ytime

def DataFit(X, Y):
    # 模型数据准备
    # X_parameter=[]
    # Y_parameter=[]
    # for single_square_feet ,single_price_value in zip(X,Y):
    #     X_parameter.append([float(single_square_feet)])
    #     Y_parameter.append(float(single_price_value))

    # # 模型拟合
    # regr = linear_model.LinearRegression()
    # regr.fit(X_parameter, Y_parameter)

    #加p_value
    try:
        data = Y
        # print(data)
        results = powerlaw.Fit(data)
        alpha = results.power_law.alpha
        xmin = results.power_law.xmin
        R, p_value = results.distribution_compare('power_law', 'lognormal')
        print("p_value:", p_value, "item alpha:", alpha, "item xmin:", xmin)
    except:
        pass

    # 模型结果与得分
    # print("#", year, ":")
    # print("幂律指数𝜑: \n", 0-regr.coef_[0])
    # print("Intercept:\n",regr.intercept_)
    # The mean square error
    # print("rest: %.8f" % np.mean((regr.predict(X_parameter) - Y_parameter) ** 2))  # 残差平方和

    # phi = 0-regr.coef_[0]
    # return phi#, p_value, alpha

def data_time():
    for csv in ['ratings_CDs_and_Vinyl', 'ratings_Books', 'ratings_Electronics', 'ratings_Movies_and_TV']:
        csv = csv.replace('ratings_', '')
        filename_in = '../RealDatasetValidation/data/amazon/' + csv +'.txt'
        filename_out = '../RealDatasetValidation/data/amazon/' + csv +'_dump.txt'
        print("********************" + csv + "*******************")
        print("input", filename_in)
        print("output", filename_out)
        sort_by_time(filename_in, filename_out)
        print("********************END" + csv + "*******************")

In [16]:
# 22507155 book
# 7824482 electronic
# 4607047 movie
# 3749004 CD

def evol():
    csv = ['ratings_CDs_and_Vinyl', 'ratings_Books', 'ratings_Electronics', 'ratings_Movies_and_TV']
    length = [3749004, 22507155, 7824482, 4607047]
    for index in range(len(csv)):
        filename_in = '../RealDatasetValidation/data/amazon/' + csv[index].replace('ratings_','') +'_dump.txt'
        for i in range(11):
            if i!=0:
                graph_main(filename_in, i, int(length[index]/10))
                print("")

In [17]:
if __name__=="__main__":
    # data_processing()
    # data_time()
    evol()

stoptime: 374900 pace: 374900
Start reading from ../RealDatasetValidation/data/amazon/CDs_and_Vinyl_dump.txt
ytime 20010801


Values less than or equal to 0 in data. Throwing out 0 or negative values
Calculating best minimal value for power law fit
/home/daven/.local/lib/python3.6/site-packages/powerlaw.py:700: RuntimeWarning: invalid value encountered in true_divide
  (Theoretical_CDF * (1 - Theoretical_CDF))
/home/daven/.local/lib/python3.6/site-packages/powerlaw.py:1605: RuntimeWarning: invalid value encountered in true_divide
  CDF = CDF/norm
'nan' in fit cumulative distribution values.
Likely underflow or overflow error: the optimal fit for this distribution gives values that are so extreme that we lack the numerical precision to calculate them.


p_value: 0.43439079627232435 item alpha: 1.534353683701113 item xmin: 2.0

stoptime: 749800 pace: 374900
Start reading from ../RealDatasetValidation/data/amazon/CDs_and_Vinyl_dump.txt
ytime 20030811


Values less than or equal to 0 in data. Throwing out 0 or negative values
Calculating best minimal value for power law fit


p_value: 0.8437815164230654 item alpha: 1.527527902585829 item xmin: 2.0

stoptime: 1124700 pace: 374900
Start reading from ../RealDatasetValidation/data/amazon/CDs_and_Vinyl_dump.txt
1000000
ytime 20050309


...

p_value: 0.8301154008702956 item alpha: 1.6190400995447982 item xmin: 3.0

stoptime: 4146336 pace: 460704
Start reading from ../RealDatasetValidation/data/amazon/Movies_and_TV_dump.txt
1000000
2000000
3000000
4000000
ytime 20140320


Values less than or equal to 0 in data. Throwing out 0 or negative values
Calculating best minimal value for power law fit


p_value: 0.31805186798841734 item alpha: 1.6233044974552335 item xmin: 6.0

stoptime: 4607040 pace: 460704
Start reading from ../RealDatasetValidation/data/amazon/Movies_and_TV_dump.txt
1000000
2000000
3000000
4000000
ytime 20140723


Values less than or equal to 0 in data. Throwing out 0 or negative values
Calculating best minimal value for power law fit


p_value: 0.9097421870306927 item alpha: 1.6338232237620063 item xmin: 3.0

